As starting point we take a SpatialData `.zarr` file, and preprocess it, so it can be visualized by Vitessce.

A SpatialData `.zarr` file can be created e.g. as follows:

```
import os

import pooch
from spatialdata_io import visium_hd

from harpy.datasets import get_registry

registry = get_registry()
unzip_path = registry.fetch(
    "transcriptomics/visium_hd/mouse/visium_hd_mouse_small_intestine.zip",
    processor=pooch.Unzip(),
)

# path to a visium experiment
path = os.path.commonpath(unzip_path)

sdata = visium_hd(
    path=path, bin_size=[ 2,8,16 ], dataset_id="Visium_HD_Mouse_Small_Intestine", bins_as_squares=False,)

for table_layer in [*sdata.tables]:
    adata = sdata[table_layer]
    adata.var_names_make_unique()
    adata.X = adata.X.tocsc()


sdata.write( ... )
```

In [ ]:
import os

from spatialdata import read_zarr

# This works, now check with large image (e.g. vizgen)

from vitessce.data_utils import (
    VAR_CHUNK_SIZE,
    multiplex_img_to_ome_zarr,
    optimize_adata,
)

data_dir = "/Users/arnedf/VIB/vitessce_spatialdata/data" # change this

spatialdata_filepath = os.path.join(data_dir, "sdata_visium_hd_temp.zarr" ) 
output_path_adata = os.path.join( data_dir, "adata_visium_hd.zarr" )
output_path_img = os.path.join( data_dir, "visium_hd.ome.zarr" )

sdata = read_zarr( spatialdata_filepath )

Create adata and ome zarr files for vitessce

In [2]:
adata=sdata[ "square_016um" ]
num_cells = adata.obs.shape[0]

In [3]:
import numpy as np

def to_square(x, y, side):
    """
    Convert an (x, y) coordinate to a polygon (square) with a given side length.
    """
    r = side / 2
    return np.array([
        [x - r, y + r],
        [x + r, y + r],
        [x + r, y - r],
        [x - r, y - r]
    ])

adata.obsm["segmentations"] = np.zeros((num_cells, 4, 2))
radius = float(sdata[ "Visium_HD_Mouse_Small_Intestine_square_016um" ]["radius"].values[0])
for i in range(num_cells):
    adata.obsm["segmentations"][i, :, :] = to_square(
        adata.obsm["spatial"][i, 0], adata.obsm["spatial"][i, 1], radius*2
    )

In [ ]:
adata = optimize_adata(
    adata,
    #obs_cols=["location_id"],
    obsm_keys=[ "segmentations", "spatial" ],
    optimize_X=True,
    # Vitessce plays nicely with dense matrices saved with chunking
    # and this one is small enough that dense is not a huge overhead.
    to_dense_X=True,
)
adata.write_zarr(output_path_adata, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

In [6]:
array=sdata["Visium_HD_Mouse_Small_Intestine_full_image" ][ "scale0" ][ "image" ].data.compute()
multiplex_img_to_ome_zarr( img_arr=array, channel_names=[ "r", "g", "b" ], axes="cyx",  output_path=output_path_img)

dask.array<from-zarr, shape=(3, 21943, 23618), dtype=uint8, chunksize=(3, 256, 256), chunktype=numpy.ndarray>

Finally, copy created `.zarr` files to bucket. E.g. to https://objectstor.vib.be/spatial-hackathon-public/_test_vitessce_1/A/1/... for instance using Globus.